In [1]:
# get all keyword data fro the given campaig
# Clean data, remove duplicates and fill null values
# for each keyword, create a row that shows the last 4 weeks perfromance
# cluster keywords based on the KPIs
# create 3 clusters, low, high and moderate campaigns

In [2]:
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
import numpy as np

In [3]:
credentials = service_account.Credentials.from_service_account_file("../certs/bq_service_account_mo.json")
client = bigquery.Client(credentials=credentials, project="bigqpr" )

In [28]:
def ad_keyword_level():
    project_id = ''
    dataset_id = ''

    table_id = '' #'apple_asa_ios_app_all_keywords'

    query = f"""
            SELECT distinct * FROM `{project_id}.{dataset_id}.{table_id}`
            WHERE date >= '2022-11-01'
            AND ad_group_id in (1260716647,
            1260714780,
            1260715395,
            1260715395, 
            1260715011,
            1260715158,
            1260715096,
            1260715157,
            1260716032,
            1260715254,
            1260715069,
            1260715774,
            1260717168,
            1260715510,
            1260715394
            )

            """

    query_job = client.query(query)

    df_keyword = query_job.to_dataframe()
    return df_keyword

In [19]:
df = ad_group_level()

In [408]:
df_keyword = ad_keyword_level()

# Keyword level data

In [409]:
df_keyword.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111675 entries, 0 to 111674
Data columns (total 30 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   account_email           111675 non-null  object        
 1   account_org_id          111675 non-null  int64         
 2   date                    111675 non-null  object        
 3   app_adam_id             111675 non-null  int64         
 4   campaign_id             111675 non-null  int64         
 5   campaign_name           111675 non-null  object        
 6   ad_group_id             111675 non-null  int64         
 7   ad_group_name           111675 non-null  object        
 8   currency                111675 non-null  object        
 9   keyword_id              111675 non-null  object        
 10  keyword                 111675 non-null  object        
 11  modification_time       111675 non-null  datetime64[ns]
 12  country_or_region       111675

In [410]:
# turn date to datetime

df_keyword["date"] = pd.to_datetime(df_keyword["date"], format="%Y-%m-%d")

In [411]:
duplicate_count = len(df_keyword) - len(df_keyword.drop_duplicates()) 

# Original data length minus data length without duplicates
duplicate_count

0

In [412]:
# view duplicate rows
df_keyword[df_keyword.duplicated(keep=False )].sort_values(by=["date"])


,account_email,account_org_id,date,app_adam_id,campaign_id,campaign_name,ad_group_id,ad_group_name,currency,keyword_id,...,impressions,installs,installs_redownloads,installs_new_downloads,lat_on_installs,lat_off_installs,avg_cpt,avg_cpa,other,deleted


In [413]:
# check if all keywords have ids
df_keyword["keyword_id"].isnull().sum()

0

In [414]:
# check if any campaign_ids are missing
df_keyword["campaign_id"].isnull().sum()

0

In [415]:
# percentage of missing data per column
round(df_keyword.isnull().sum().sort_values(ascending=False)/len(df_keyword),2) #NaN percentage for each column


account_email             0.0
account_org_id            0.0
other                     0.0
avg_cpa                   0.0
avg_cpt                   0.0
lat_off_installs          0.0
lat_on_installs           0.0
installs_new_downloads    0.0
installs_redownloads      0.0
installs                  0.0
impressions               0.0
local_spend               0.0
taps                      0.0
ttr                       0.0
bid                       0.0
keyword_display_status    0.0
keyword_status            0.0
match_type                0.0
country_or_region         0.0
modification_time         0.0
keyword                   0.0
keyword_id                0.0
currency                  0.0
ad_group_name             0.0
ad_group_id               0.0
campaign_name             0.0
campaign_id               0.0
app_adam_id               0.0
date                      0.0
deleted                   0.0
dtype: float64

In [416]:
df_keyword.columns

Index(['account_email', 'account_org_id', 'date', 'app_adam_id', 'campaign_id',
       'campaign_name', 'ad_group_id', 'ad_group_name', 'currency',
       'keyword_id', 'keyword', 'modification_time', 'country_or_region',
       'match_type', 'keyword_status', 'keyword_display_status', 'bid', 'ttr',
       'taps', 'local_spend', 'impressions', 'installs',
       'installs_redownloads', 'installs_new_downloads', 'lat_on_installs',
       'lat_off_installs', 'avg_cpt', 'avg_cpa', 'other', 'deleted'],
      dtype='object')

In [417]:
df_keyword["keyword_display_status"].unique()

array(['RUNNING', 'PAUSED'], dtype=object)

In [418]:
df_keyword["keyword_display_status"].unique()

array(['RUNNING', 'PAUSED'], dtype=object)

In [419]:
df_keyword["keyword_status"].unique()

array(['ACTIVE', 'PAUSED'], dtype=object)

In [420]:
df_keyword["match_type"].unique()

array(['EXACT', 'BROAD'], dtype=object)

In [421]:
df_keyword["currency"].unique()

array(['0'], dtype=object)

In [422]:
df_keyword["country_or_region"].unique()

array(['MX'], dtype=object)

In [423]:
df_keyword[(df_keyword["keyword_status"]=="PAUSED") & (df_keyword["impressions"] ==0)]

,account_email,account_org_id,date,app_adam_id,campaign_id,campaign_name,ad_group_id,ad_group_name,currency,keyword_id,...,impressions,installs,installs_redownloads,installs_new_downloads,lat_on_installs,lat_off_installs,avg_cpt,avg_cpa,other,deleted
28,Phiture US,4958460,2023-08-17,469337564,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,0,1260719071,...,0,0,0,0,0,0.0,0.0,0.0,False,False
45,Phiture US,4958460,2023-08-19,469337564,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,0,1260719077,...,0,0,0,0,0,0.0,0.0,0.0,False,False
73,Phiture US,4958460,2023-08-18,469337564,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,0,1260719066,...,0,0,0,0,0,0.0,0.0,0.0,False,False
75,Phiture US,4958460,2023-01-10,469337564,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,0,1260719066,...,0,0,0,0,0,0.0,0.0,0.0,False,False
115,Phiture US,4958460,2023-03-18,469337564,1260482209,MX_Reader_ASA_Generic_ES-EN_Q1,1260715157,MX_Reader_ASA_Generic_Read_ES_Q1,0,1260718084,...,0,0,0,0,0,0.0,0.0,0.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111600,Phiture US,4958460,2022-12-19,469337564,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,0,1260719081,...,0,0,0,0,0,0.0,0.0,0.0,False,False
111604,Phiture US,4958460,2023-03-25,469337564,1260482209,MX_Reader_ASA_Generic_ES-EN_Q1,1260715157,MX_Reader_ASA_Generic_Read_ES_Q1,0,1260718084,...,0,0,0,0,0,0.0,0.0,0.0,False,False
111648,Phiture US,4958460,2023-06-27,469337564,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,0,1260719079,...,0,0,0,0,0,0.0,0.0,0.0,False,False
111649,Phiture US,4958460,2023-06-27,469337564,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,0,1260719071,...,0,0,0,0,0,0.0,0.0,0.0,False,False


In [425]:
df_keyword[df_keyword["keyword_status"]=="PAUSED"]["keyword_id"].nunique()

20

In [49]:
df_keyword[df_keyword["keyword_status"]=="PAUSED"].describe()

,account_org_id,app_adam_id,campaign_id,ad_group_id,modification_time,bid,ttr,taps,local_spend,impressions,installs,installs_redownloads,installs_new_downloads,lat_on_installs,lat_off_installs,avg_cpt,avg_cpa
count,5780.0,5780.0,5.780000e+03,5.780000e+03,5780,5780.000000,5780.000000,5780.000000,5780.000000,5780.000000,5780.000000,5780.000000,5780.000000,5780.0,5780.0,5780.000000,5780.000000
mean,4958460.0,469337564.0,1.260487e+09,1.260715e+09,2023-05-14 14:43:27.571430144,7.939339,0.000326,0.007958,0.002206,0.108651,0.003633,0.001211,0.002422,0.0,0.0,0.000946,0.001668
min,4958460.0,469337564.0,1.260482e+09,1.260715e+09,2023-04-21 00:20:55.727000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
25%,4958460.0,469337564.0,1.260487e+09,1.260715e+09,2023-04-21 00:21:03.344750080,8.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
50%,4958460.0,469337564.0,1.260487e+09,1.260715e+09,2023-05-21 00:17:38.508999936,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
75%,4958460.0,469337564.0,1.260487e+09,1.260715e+09,2023-06-01 00:13:04.607000064,10.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
max,4958460.0,469337564.0,1.260487e+09,1.260715e+09,2023-08-22 00:07:27.383000,10.000000,0.222200,6.000000,1.380000,45.000000,2.000000,2.000000,2.000000,0.0,0.0,0.540000,1.380000
std,0.0,0.0,1.431124e+03,1.008730e+02,NaN,3.781531,0.006396,0.157653,0.043640,1.749386,0.073151,0.039445,0.052562,0.0,0.0,0.017522,0.036763


In [52]:
# drop all paused keywords ~25 rows
df_keyword = df_keyword[df_keyword["keyword_status"]!="PAUSED"]


In [56]:
# drop columns
df_keyword = df_keyword[['date',
             'campaign_id',
             'campaign_name',
             'ad_group_id',
             'ad_group_name',
             'keyword_id',
             'keyword',
             'country_or_region',
             'match_type',
             'keyword_status',
             'keyword_display_status',
             'bid',
             'ttr',
             'taps',
             'local_spend',
             'impressions',
             'installs',
             'avg_cpt',
             'avg_cpa',]].copy()


In [184]:
#sort by date then campaign id then ad group id, then keyword id

df_keyword_sorted = df_keyword.sort_values(by=["date", "campaign_id", "ad_group_id", "keyword_id"])

In [185]:
# add conversion_rate

df_keyword_sorted["avg_cvr"] = df_keyword_sorted["installs"]/ df_keyword_sorted["taps"]

# Check number of keywords per ad group

In [186]:
df_keyword_sorted.groupby(["campaign_id"]).nunique()["keyword_id"].sum()

469

In [187]:
df_keyword_sorted.groupby(["campaign_id", "ad_group_id"]).nunique()["keyword_id"]

campaign_id  ad_group_id
1260482209   1260715011      14
             1260715069       6
             1260715096       8
             1260715157       6
             1260715158       8
             1260715254      12
             1260715394      23
             1260715395       8
             1260715510       5
             1260715774      12
             1260716032      36
             1260717168      19
1260486979   1260714780     312
Name: keyword_id, dtype: int64

In [210]:
# fill na
df_keyword_sorted.fillna(0, inplace=True)

In [406]:
df_keyword_sorted["date"].min()

Timestamp('2022-11-07 00:00:00')

In [71]:
df_keyword_sorted.columns

Index(['date', 'campaign_id', 'campaign_name', 'ad_group_id', 'ad_group_name',
       'keyword_id', 'keyword', 'country_or_region', 'match_type',
       'keyword_status', 'keyword_display_status', 'bid', 'ttr', 'taps',
       'local_spend', 'impressions', 'installs', 'avg_cpt', 'avg_cpa',
       'avg_cvr'],
      dtype='object')

In [174]:
df_keyword_sorted["keyword_id"].nunique()

469

In [257]:
keyword_id_ids = df_keyword_sorted["keyword_id"].unique()

# loop over campaigns
df_list = []
for keyword_id in keyword_id_ids:
    df = df_keyword_sorted[df_keyword_sorted["keyword_id"]==keyword_id].copy()
    # sort df by date
    df.sort_values(by="date", ascending=True, inplace=True)
    df = df.tail(30)
    #print(df["date"].max(), df["date"].min())
    
    df_list.append(df)
    
# re merge all datasets
X_fe = pd.concat(df_list, axis=0)
    

In [260]:
mask = X_fe.groupby("keyword_id")["impressions"].sum() > 10

# only keep keywords with > 0 impressions in the past 30 days
X_subset = X_fe.groupby("keyword_id")["impressions"].sum()[mask]
X_subset = X_fe[X_fe["keyword_id"].isin(X_subset.index)]

In [261]:
X_subset

,date,campaign_id,campaign_name,ad_group_id,ad_group_name,keyword_id,keyword,country_or_region,match_type,keyword_status,keyword_display_status,bid,ttr,taps,local_spend,impressions,installs,avg_cpt,avg_cpa,avg_cvr
102952,2023-07-24,1260482209,MX_Reader_ASA_Generic_ES-EN_Q1,1260715011,MX_Reader_ASA_Generic_Fill/Write_ES_Q1,1260718812,notes pdf,MX,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
92675,2023-07-25,1260482209,MX_Reader_ASA_Generic_ES-EN_Q1,1260715011,MX_Reader_ASA_Generic_Fill/Write_ES_Q1,1260718812,notes pdf,MX,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
41638,2023-07-26,1260482209,MX_Reader_ASA_Generic_ES-EN_Q1,1260715011,MX_Reader_ASA_Generic_Fill/Write_ES_Q1,1260718812,notes pdf,MX,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
12508,2023-07-27,1260482209,MX_Reader_ASA_Generic_ES-EN_Q1,1260715011,MX_Reader_ASA_Generic_Fill/Write_ES_Q1,1260718812,notes pdf,MX,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
40219,2023-07-28,1260482209,MX_Reader_ASA_Generic_ES-EN_Q1,1260715011,MX_Reader_ASA_Generic_Fill/Write_ES_Q1,1260718812,notes pdf,MX,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102932,2023-08-18,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,1417439461,pdf fill,MX,BROAD,ACTIVE,RUNNING,0.68,0.0,0,0.0,0,0,0.0,0.0,0.0
109615,2023-08-19,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,1417439461,pdf fill,MX,BROAD,ACTIVE,RUNNING,0.68,0.0,0,0.0,0,0,0.0,0.0,0.0
66801,2023-08-20,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,1417439461,pdf fill,MX,BROAD,ACTIVE,RUNNING,0.68,0.0,0,0.0,0,0,0.0,0.0,0.0
100390,2023-08-21,1260486979,MX_Reader_ASA_Discovery_ES-EN_Q1,1260714780,MX_Reader_ASA_Discovery_Broad_ES-EN_Q1,1417439461,pdf fill,MX,BROAD,ACTIVE,RUNNING,0.68,0.0,0,0.0,0,0,0.0,0.0,0.0


In [231]:
# annonymize data
import uuid


campaign_ids = X_fe["campaign_id"].unique()
ad_group_ids = X_fe["ad_group_id"].unique()
keyword_ids = X_fe["keyword_id"].unique()

c_id_dict = {}
ad_id_dict = {}
key_id_dict = {}


for c_id in campaign_ids:
    new_id = str(uuid.uuid4())
    c_id_dict[c_id] = new_id

for ad_id in ad_group_ids:
    new_ad_name = str(uuid.uuid4())
    ad_id_dict[ad_id] = new_ad_name
    

for key_id in keyword_ids:
    new_key_name = str(uuid.uuid4())
    key_id_dict[key_id] = new_key_name
    
    
df_moc = X_fe.copy()

df_moc["campaign_id"] = df_moc["campaign_id"].map(c_id_dict)
df_moc["ad_group_id"] = df_moc["ad_group_id"].map(ad_id_dict)
df_moc["keyword_id"] = df_moc["keyword_id"].map(key_id_dict)



In [232]:
df_moc.drop(columns=["campaign_name",
                   "ad_group_name",
                   "keyword",
                   "country_or_region"], inplace=True)

In [233]:
df_moc.to_csv("df_moc_keyword.csv")

In [235]:
df_moc

,date,campaign_id,ad_group_id,keyword_id,match_type,keyword_status,keyword_display_status,bid,ttr,taps,local_spend,impressions,installs,avg_cpt,avg_cpa,avg_cvr
91578,2023-05-25,73d7f036-42bc-4e51-939f-a125ae0b0d30,d4dbb962-5daa-4cf6-b82b-eef3882aa1a3,c58d2d9c-88bc-4176-9405-66cd49a72b06,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
56411,2023-05-26,73d7f036-42bc-4e51-939f-a125ae0b0d30,d4dbb962-5daa-4cf6-b82b-eef3882aa1a3,c58d2d9c-88bc-4176-9405-66cd49a72b06,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
78295,2023-05-27,73d7f036-42bc-4e51-939f-a125ae0b0d30,d4dbb962-5daa-4cf6-b82b-eef3882aa1a3,c58d2d9c-88bc-4176-9405-66cd49a72b06,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
3226,2023-05-28,73d7f036-42bc-4e51-939f-a125ae0b0d30,d4dbb962-5daa-4cf6-b82b-eef3882aa1a3,c58d2d9c-88bc-4176-9405-66cd49a72b06,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
40570,2023-05-29,73d7f036-42bc-4e51-939f-a125ae0b0d30,d4dbb962-5daa-4cf6-b82b-eef3882aa1a3,c58d2d9c-88bc-4176-9405-66cd49a72b06,EXACT,ACTIVE,RUNNING,10.00,0.0,0,0.0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83341,2023-08-18,3ec1eec6-525a-4a90-83e6-0eab554e9571,8bf6befa-d2fb-4386-a440-1748fa507ba8,f8e8c37c-e904-4831-9979-6ab07a0facbe,BROAD,ACTIVE,RUNNING,0.68,0.0,0,0.0,0,0,0.0,0.0,0.0
93998,2023-08-19,3ec1eec6-525a-4a90-83e6-0eab554e9571,8bf6befa-d2fb-4386-a440-1748fa507ba8,f8e8c37c-e904-4831-9979-6ab07a0facbe,BROAD,ACTIVE,RUNNING,0.68,0.0,0,0.0,0,0,0.0,0.0,0.0
58997,2023-08-20,3ec1eec6-525a-4a90-83e6-0eab554e9571,8bf6befa-d2fb-4386-a440-1748fa507ba8,f8e8c37c-e904-4831-9979-6ab07a0facbe,BROAD,ACTIVE,RUNNING,0.68,0.0,0,0.0,0,0,0.0,0.0,0.0
12295,2023-08-21,3ec1eec6-525a-4a90-83e6-0eab554e9571,8bf6befa-d2fb-4386-a440-1748fa507ba8,f8e8c37c-e904-4831-9979-6ab07a0facbe,BROAD,ACTIVE,RUNNING,0.68,0.0,0,0.0,1,0,0.0,0.0,0.0


# Clustering algorithm

In [212]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [213]:
df_keyword_sorted.columns

Index(['date', 'campaign_id', 'campaign_name', 'ad_group_id', 'ad_group_name',
       'keyword_id', 'keyword', 'country_or_region', 'match_type',
       'keyword_status', 'keyword_display_status', 'bid', 'ttr', 'taps',
       'local_spend', 'impressions', 'installs', 'avg_cpt', 'avg_cpa',
       'avg_cvr'],
      dtype='object')

In [214]:
df_keyword_sorted["keyword_id"].nunique()

469

In [426]:
# Convert the 'date' column to datetime format
df = df_keyword_sorted.copy()

# Get the most recent date for each keyword
latest_date_per_keyword = df.groupby('keyword_id')['date'].max()

# Filter out rows that are older than 60 days from the most recent date for each keyword
filtered_rows = []
for keyword_id, latest_date in latest_date_per_keyword.items():
    start_date = latest_date - pd.Timedelta(days=60)
    keyword_rows = df[(df['keyword_id'] == keyword_id) & (df['date'] >= start_date) & (df['date'] <= latest_date)]
    filtered_rows.append(keyword_rows)

filtered_df = pd.concat(filtered_rows, axis=0)



In [433]:
filtered_df["keyword_id"].nunique()

469

In [432]:
filtered_df.groupby("keyword_id")["date"].agg(["max", "min"])

,max,min
keyword_id,,
1260714837,2023-08-22,2023-06-23
1260714838,2023-08-22,2023-06-23
1260714839,2023-08-22,2023-06-23
1260714840,2023-08-22,2023-06-23
1260714979,2023-08-22,2023-06-23
...,...,...
1417439475,2023-08-22,2023-07-08
1417439476,2023-08-22,2023-07-08
1417439477,2023-08-22,2023-07-08


In [434]:
filtered_df = filtered_df[filtered_df["impressions"] > 1]

In [435]:
filtered_df["keyword_id"].nunique()

137

In [364]:
# Aggregate the filtered data at the keyword level
agg_recent_df = filtered_df.groupby('keyword_id').agg({
    'ttr': 'mean',
    'avg_cpt': 'mean',
    'avg_cpa': 'mean',
    'local_spend': ['mean', 'sum'],
    'impressions': ['mean', 'sum'],
    'installs': ['mean', 'sum'],
    'taps': ['mean', 'sum'],
    'bid': ['mean', 'sum'],
}).reset_index()

agg_recent_df


agg_recent_df.columns = list(map('_'.join, agg_recent_df.columns.values))


agg_recent_df["ttr_mean"] = agg_recent_df["taps_sum"]/ agg_recent_df["impressions_sum"]
agg_recent_df["avg_cvr_mean"] = agg_recent_df["installs_sum"]/ agg_recent_df["taps_sum"]
agg_recent_df["avg_cpa_mean"] = agg_recent_df["local_spend_sum"]/ agg_recent_df["installs_sum"]
agg_recent_df["avg_cpt_mean"] = agg_recent_df["local_spend_sum"]/ agg_recent_df["taps_sum"]

agg_recent_df.fillna(0, inplace = True)
# drop infinite values
agg_recent_df = agg_recent_df[~agg_recent_df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]


In [384]:
features = ["local_spend_sum", "ttr_mean", "avg_cpa_mean", "avg_cpt_mean", "avg_cvr_mean"]

# Scaling the features
scaler = StandardScaler()

scaled_features = scaler.fit_transform(agg_recent_df[features])


# Applying KMeans clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

# Adding the cluster labels to the aggregated dataframe
agg_recent_df['cluster'] = clusters




In [385]:
agg_recent_df.groupby('cluster')["keyword_id_"].nunique()

cluster
0    28
1     7
2    98
Name: keyword_id_, dtype: int64

In [ ]:
cluster_recent_means.drop(columns="keyword_id_").reset_index().to_csv("clusters.csv")

Cluster 0:

Lowest Tap Through Rate (TTR): This indicates that keywords in this cluster have the lowest click-through rate.
Lowest Cost Per Tap (CPT) and Cost Per Acquisition (CPA): Advertisers pay less for each tap and acquisition for these keywords.
Lowest average and total spend: This could mean these keywords might be less competitive or less popular.
Lowest impressions (both average and total): These keywords are not shown to users as frequently as those in other clusters.
Lowest installs (both average and total): These keywords lead to the fewest app installs.
Low taps (both average and total): Keywords in this cluster receive the least interaction.
Highest average bid, but lowest total bid: Advertisers might be willing to pay more for each bid, but the total amount spent on these keywords is the lowest.
Lowest Conversion Rate (CVR): The rate of conversion from impressions to installs is the lowest.
Cluster 1:
Medium Tap Through Rate (TTR): Keywords in this cluster have a moderate click-through rate.
Highest Cost Per Tap (CPT) and Cost Per Acquisition (CPA): Advertisers pay the most for each tap and acquisition for these keywords.
Highest average and total spend: Keywords in this cluster are likely highly competitive or popular.
Highest impressions (both average and total): These keywords are shown to users most frequently.
Medium installs and taps (both average and total): These keywords lead to a moderate number of app installs and interactions.
Highest total bid, but a medium average bid: Advertisers spend the most overall on these keywords, but each bid is moderate in value.
Medium Conversion Rate (CVR): The rate of conversion from impressions to installs is moderate.
Cluster 2:
Highest Tap Through Rate (TTR): Keywords in this cluster have the best click-through rate.
Medium Cost Per Tap (CPT) and Cost Per Acquisition (CPA): Advertisers pay a moderate amount for each tap and acquisition for these keywords.
Medium average and total spend: This could indicate a balanced competition or popularity level for these keywords.
Medium impressions (both average and total): These keywords are shown to users a moderate number of times.
Highest installs (both average and total): These keywords lead to the most app installs.
Highest taps (both average and total): Keywords in this cluster receive the most interaction.
Lowest average and total bid: Even though these keywords perform well, advertisers pay the least for each bid and overall.
Highest Conversion Rate (CVR): The rate of conversion from impressions to installs is the highest.
Based on the above analysis, we can infer:

Cluster 0: These keywords might be "Low Engagement & Low Cost" keywords.

Cluster 1: These keywords could be "High Visibility & High Cost" keywords.

Cluster 2: These keywords appear to be "High Performance & Low Cost" keywords.

In [387]:
# Adding cluster names
cluster_names = {0 :"Low Engagement & Low Costs", 1: "High Visibility & High Cos", 2:"High Performance & Low Cost"}
agg_recent_df['cluster_name'] = agg_recent_df["cluster"].map(cluster_names)

In [388]:
agg_recent_df.head()

,keyword_id_,ttr_mean,avg_cpt_mean,avg_cpa_mean,local_spend_mean,local_spend_sum,impressions_mean,impressions_sum,installs_mean,installs_sum,taps_mean,taps_sum,bid_mean,bid_sum,avg_cvr_mean,cluster,Cluster Name,cluster_name
0,1260715297,0.272451,0.320721,0.421064,15.000404,720.0194,171.666667,8240,35.625000,1710,46.770833,2245,0.335208,16.09,0.761693,2,High Performance & Low Cost,High Performance & Low Cost
1,1260715298,0.500000,2.659550,5.319100,2.659550,5.3191,2.000000,4,0.500000,1,1.000000,2,1.905000,3.81,0.500000,1,High Visibility & High Cos,High Visibility & High Cos
2,1260715459,0.000000,0.000000,0.000000,0.000000,0.0000,2.000000,2,0.000000,0,0.000000,0,0.050000,0.05,0.000000,0,Low Engagement & Low Costs,Low Engagement & Low Costs
3,1260715873,0.023810,4.567250,9.134500,0.314983,9.1345,2.896552,84,0.034483,1,0.068966,2,5.744138,166.58,0.500000,1,High Visibility & High Cos,High Visibility & High Cos
4,1260715876,0.058824,1.259400,2.518800,0.359829,2.5188,4.857143,34,0.142857,1,0.285714,2,1.092857,7.65,0.500000,2,High Performance & Low Cost,High Performance & Low Cost


In [405]:
# Picking top 3 keywords with highest impressions for each cluster
top_keywords = []

for cluster_num, cluster_name in enumerate(cluster_names):
    top_3 = agg_recent_df[agg_recent_df['cluster'] == cluster_num].nlargest(5, 'impressions_sum')
    top_keywords.append(top_3)

top_keywords_df = pd.concat(top_keywords, axis=0)

top_keywords_df = top_keywords_df[['keyword_id_',
                 'cluster',
                 'cluster_name',
                 'impressions_sum',
                 "taps_sum",
                 'installs_sum',
                 "ttr_mean",
                 "avg_cpa_mean", 
                "avg_cpt_mean",
                "avg_cvr_mean"]]

top_keywords_df

,keyword_id_,cluster,cluster_name,impressions_sum,taps_sum,installs_sum,ttr_mean,avg_cpa_mean,avg_cpt_mean,avg_cvr_mean
88,1260719006,0,Low Engagement & Low Costs,2973,69,30,0.023209,0.599767,0.260768,0.434783
124,1417439414,0,Low Engagement & Low Costs,296,9,3,0.030405,0.492367,0.164122,0.333333
89,1260719008,0,Low Engagement & Low Costs,210,7,2,0.033333,1.012900,0.289400,0.285714
91,1260719016,0,Low Engagement & Low Costs,102,7,3,0.068627,0.379833,0.162786,0.428571
133,1417439471,0,Low Engagement & Low Costs,73,8,3,0.109589,0.434733,0.163025,0.375000
9,1260716572,1,High Visibility & High Cos,10161,165,88,0.016239,3.165619,1.688330,0.533333
20,1260718823,1,High Visibility & High Cos,7829,160,97,0.020437,3.621454,2.195506,0.606250
25,1260718831,1,High Visibility & High Cos,4435,110,62,0.024803,6.534700,3.683195,0.563636
24,1260718830,1,High Visibility & High Cos,1073,27,14,0.025163,8.506857,4.410963,0.518519
3,1260715873,1,High Visibility & High Cos,84,2,1,0.023810,9.134500,4.567250,0.500000


In [401]:
top_keywords_df.to_csv("top_keywords.csv")